# Example of `InferenceData` schema in PyStan
The description of the `InferenceData` structure can be found in [GitHub](https://github.com/arviz-devs/arviz/blob/master/schema.md).

In [1]:
import arviz as az
import pystan
import pandas as pd
import numpy as np
import xarray
# xarray.set_options(display_style="html");

In [2]:
#read data
data = pd.read_csv("linear_regression_data.csv", index_col=0)
time_since_joined = data.time.values
slack_comments = data.comments.values
names = data.index.values
N = len(names)
data

,comments,time
Alice,7500,4.5
Bob,10100,6.0
Cole,18600,7.0
Danielle,25200,12.0
Erika,27500,18.0


In [3]:
linreg_prior_code = """
data {
  int<lower=0> N;
  real time_since_joined[N];
}

generated quantities {
    real b0;
    real b1;
    real log_sigma;
    real<lower=0> sigma;
    vector[N] slack_comments_hat;
    
    b0 = normal_rng(0,10);
    b1 = normal_rng(0,10);
    sigma = abs(normal_rng(0,30));
    log_sigma = log(sigma);
    for (i in 1:N) {
        slack_comments_hat[i] = normal_rng(b0 + b1 * time_since_joined[i], sigma);
    }
}
"""
sm_prior = pystan.StanModel(model_code=linreg_prior_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_b2d530337b2d60d5322cb10cbc01e530 NOW.


In [4]:
linreg_prior_data_dict = {"N": N, "time_since_joined": time_since_joined}
prior = sm_prior.sampling(data=linreg_prior_data_dict, iter=150, chains=1, algorithm='Fixed_param', warmup=0)

In [5]:
linreg_code = """
data {
  int<lower=0> N;
  real time_since_joined[N];
  real slack_comments[N];
}

parameters {
  real b0;
  real b1;
  real log_sigma;
}

transformed parameters {
  real<lower=0> sigma;
  sigma = exp(log_sigma);
}

model {
  b0 ~ normal(0,10);
  b1 ~ normal(0,10);
  sigma ~ normal(0,30);
  for (i in 1:N) {
    slack_comments[i] ~ normal(b0 + b1 * time_since_joined[i], sigma);
  }
  
}

generated quantities {
    vector[N] log_lik;
    vector[N] slack_comments_hat;
    for (i in 1:N) {
        log_lik[i] = normal_lpdf(slack_comments[i] | b0 + b1 * time_since_joined[i], sigma);
        slack_comments_hat[i] = normal_rng(b0 + b1 * time_since_joined[i], sigma);
    }
}
"""
sm = pystan.StanModel(model_code=linreg_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_13939a7ccc2875c5ec3f9d016bdf59f1 NOW.


In [6]:
linreg_data_dict = {"N": N, "slack_comments": slack_comments, "time_since_joined": time_since_joined}
posterior = sm.sampling(data=linreg_data_dict, iter=200, chains=4)

In [7]:
idata_stan = az.from_pystan(
    posterior=posterior,
    prior=prior,
    posterior_predictive="slack_comments_hat",
    prior_predictive="slack_comments_hat",
    observed_data=["slack_comments"],
    constant_data=["time_since_joined"],
    log_likelihood="log_lik",
    coords={"developer": names},
    dims={
        "slack_comments": ["developer"],
        "log_lik": ["developer"],
        "slack_comments_hat": ["developer"],
        "time_since_joined": ["developer"],
    }
)

In [8]:
idata_stan

Inference data with groups:
	> posterior
	> sample_stats
	> posterior_predictive
	> prior
	> sample_stats_prior
	> prior_predictive
	> observed_data
	> constant_data

In this example, each variable has as dimension a combination of the following 3: `chain`, `draw` and `developer`. Moreover, each dimension has specific coordinate values. In the case of `chain` and `draw` it is an integer identifier starting at `0`; in the case of `developer` dimension, its coordinate values are the following strings: `["Alice", "Bob", "Cole", "Danielle", "Erika"]`.

In [9]:
idata_stan.posterior

<xarray.Dataset>
Dimensions:    (chain: 4, draw: 100)
Coordinates:
  * chain      (chain) int64 0 1 2 3
  * draw       (draw) int64 0 1 2 3 4 5 6 7 8 9 ... 91 92 93 94 95 96 97 98 99
Data variables:
    b0         (chain, draw) float64 -4.608 -1.423 -4.129 ... 4.576 -0.443 5.092
    b1         (chain, draw) float64 77.9 73.76 70.86 ... 76.75 81.73 72.28
    log_sigma  (chain, draw) float64 7.029 7.007 7.017 ... 7.026 7.025 7.003
    sigma      (chain, draw) float64 1.129e+03 1.104e+03 ... 1.125e+03 1.1e+03
Attributes:
    created_at:                 2019-11-14T23:28:01.343823
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [10]:
idata_stan.sample_stats

<xarray.Dataset>
Dimensions:         (chain: 4, developer: 5, draw: 100)
Coordinates:
  * chain           (chain) int64 0 1 2 3
  * draw            (draw) int64 0 1 2 3 4 5 6 7 8 ... 92 93 94 95 96 97 98 99
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    accept_stat     (chain, draw) float64 0.9028 0.8348 0.9991 ... 0.9895 0.9044
    stepsize        (chain, draw) float64 0.01667 0.01667 ... 0.01547 0.01547
    treedepth       (chain, draw) int64 10 10 9 10 10 10 10 ... 10 10 10 2 10 10
    n_leapfrog      (chain, draw) int64 1023 1023 883 1023 ... 1023 3 1023 1023
    diverging       (chain, draw) bool False False False ... False False False
    energy          (chain, draw) float64 1.46e+03 1.458e+03 ... 1.458e+03
    log_likelihood  (chain, draw, developer) float64 -28.03 -44.39 ... -291.6
    lp              (chain, draw) float64 -1.456e+03 -1.456e+03 ... -1.457e+03
Attributes:
    created_at:                 2019-11-14T23:28:01.353126
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [11]:
idata_stan.posterior_predictive

<xarray.Dataset>
Dimensions:             (chain: 4, developer: 5, draw: 100)
Coordinates:
  * chain               (chain) int64 0 1 2 3
  * draw                (draw) int64 0 1 2 3 4 5 6 7 ... 92 93 94 95 96 97 98 99
  * developer           (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    slack_comments_hat  (chain, draw, developer) float64 -1.232e+03 ... 539.4
Attributes:
    created_at:                 2019-11-14T23:28:01.358833
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [12]:
idata_stan.observed_data

<xarray.Dataset>
Dimensions:         (developer: 5)
Coordinates:
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (developer) int64 7500 10100 18600 25200 27500
Attributes:
    created_at:                 2019-11-14T23:28:01.335241
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [13]:
idata_stan.constant_data

<xarray.Dataset>
Dimensions:            (developer: 5)
Coordinates:
  * developer          (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    time_since_joined  (developer) float64 4.5 6.0 7.0 12.0 18.0
Attributes:
    created_at:                 2019-11-14T23:28:01.339140
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [14]:
idata_stan.prior

<xarray.Dataset>
Dimensions:    (chain: 1, draw: 150)
Coordinates:
  * chain      (chain) int64 0
  * draw       (draw) int64 0 1 2 3 4 5 6 7 ... 142 143 144 145 146 147 148 149
Data variables:
    b0         (chain, draw) float64 15.08 -0.5347 5.759 ... 6.636 16.74 10.35
    b1         (chain, draw) float64 0.7568 -6.876 -9.222 ... 16.71 5.283 4.06
    log_sigma  (chain, draw) float64 2.337 2.949 2.045 ... 2.123 3.229 2.64
    sigma      (chain, draw) float64 10.35 19.09 7.733 ... 8.36 25.27 14.01
Attributes:
    created_at:                 2019-11-14T23:28:01.361860
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [15]:
idata_stan.sample_stats_prior

<xarray.Dataset>
Dimensions:      (chain: 1, draw: 150)
Coordinates:
  * chain        (chain) int64 0
  * draw         (draw) int64 0 1 2 3 4 5 6 7 ... 143 144 145 146 147 148 149
Data variables:
    accept_stat  (chain, draw) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    lp           (chain, draw) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    created_at:                 2019-11-14T23:28:01.365743
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [16]:
idata_stan.prior_predictive

<xarray.Dataset>
Dimensions:             (chain: 1, developer: 5, draw: 150)
Coordinates:
  * chain               (chain) int64 0
  * draw                (draw) int64 0 1 2 3 4 5 6 ... 144 145 146 147 148 149
  * developer           (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    slack_comments_hat  (chain, draw, developer) float64 21.06 30.87 ... 76.26
Attributes:
    created_at:                 2019-11-14T23:28:01.369135
    inference_library:          pystan
    inference_library_version:  2.19.1.1